# Basic Examples with Different Protocols Showing Metrics

## Prerequisites

 * A kubernetes cluster with kubectl configured
 * curl
 * grpcurl
 * pygmentize
 

## Setup Seldon Core

Install Seldon Core as described in [docs](https://docs.seldon.io/projects/seldon-core/en/latest/workflow/install.html)

Then port-forward to that ingress on localhost:8003 in a separate terminal either with:

 * Ambassador: 
 
 ```bash
 kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
 ```
 * Istio: 
 
 ```bash
 kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8003:80
 ```
 

In [1]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "kind-ansible" modified.


## Install Seldon Analytics

In [3]:
!helm install seldon-core-analytics ../../../helm-charts/seldon-core-analytics  \
        --set grafana_prom_admin_password=password \
        --set persistence.enabled=false \
        --namespace seldon-system \
        --wait

W0401 16:50:29.277945  549535 warnings.go:70] rbac.authorization.k8s.io/v1beta1 ClusterRole is deprecated in v1.17+, unavailable in v1.22+; use rbac.authorization.k8s.io/v1 ClusterRole
W0401 16:50:29.278960  549535 warnings.go:70] rbac.authorization.k8s.io/v1beta1 ClusterRole is deprecated in v1.17+, unavailable in v1.22+; use rbac.authorization.k8s.io/v1 ClusterRole
W0401 16:50:29.280031  549535 warnings.go:70] rbac.authorization.k8s.io/v1beta1 ClusterRole is deprecated in v1.17+, unavailable in v1.22+; use rbac.authorization.k8s.io/v1 ClusterRole
W0401 16:50:29.280803  549535 warnings.go:70] rbac.authorization.k8s.io/v1beta1 ClusterRole is deprecated in v1.17+, unavailable in v1.22+; use rbac.authorization.k8s.io/v1 ClusterRole
W0401 16:50:29.283179  549535 warnings.go:70] rbac.authorization.k8s.io/v1beta1 ClusterRoleBinding is deprecated in v1.17+, unavailable in v1.22+; use rbac.authorization.k8s.io/v1 ClusterRoleBinding
W0401 16:50:29.284050  549535 warnings.go:70] rbac.authorizat

Port forward to the Grafana dashboard

```bash
kubectl port-forward $(kubectl get pods -n seldon-system  -l app.kubernetes.io/name=grafana -o jsonpath='{.items[0].metadata.name}') 3000:3000 -n seldon-system
```

In [4]:
%env RESOURCES=../../../notebooks/resources

env: RESOURCES=../../../notebooks/resources


## Seldon Protocol REST Model

**Make sure your active namespace is seldon**

In [8]:
!pygmentize ${RESOURCES}/model_seldon_rest.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: rest-seldon
spec:
  name: restseldon
  protocol: seldon
  transport: rest  
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:1.6.0-dev
          name: classifier
    graph:
      name: classifier
      type: MODEL
    name: model
    replicas: 1


In [9]:
!kubectl apply -f ${RESOURCES}/model_seldon_rest.yaml -n seldon

seldondeployment.machinelearning.seldon.io/rest-seldon unchanged


In [10]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=rest-seldon \
                                 -o jsonpath='{.items[0].metadata.name}')

deployment "rest-seldon-model-0-classifier" successfully rolled out


In [11]:
!for i in `seq 1 60`; do \
   sleep 1 && curl -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/rest-seldon/api/v1.0/predictions \
   -H "Content-Type: application/json"; \
done

{"data":{"names":["proba"],"ndarray":[[0.43782349911420193]]},"meta":{"requestPath":{"classifier":"seldonio/mock_classifier:1.6.0-dev"}}}
{"data":{"names":["proba"],"ndarray":[[0.43782349911420193]]},"meta":{"requestPath":{"classifier":"seldonio/mock_classifier:1.6.0-dev"}}}
{"data":{"names":["proba"],"ndarray":[[0.43782349911420193]]},"meta":{"requestPath":{"classifier":"seldonio/mock_classifier:1.6.0-dev"}}}
{"data":{"names":["proba"],"ndarray":[[0.43782349911420193]]},"meta":{"requestPath":{"classifier":"seldonio/mock_classifier:1.6.0-dev"}}}
{"data":{"names":["proba"],"ndarray":[[0.43782349911420193]]},"meta":{"requestPath":{"classifier":"seldonio/mock_classifier:1.6.0-dev"}}}
{"data":{"names":["proba"],"ndarray":[[0.43782349911420193]]},"meta":{"requestPath":{"classifier":"seldonio/mock_classifier:1.6.0-dev"}}}
{"data":{"names":["proba"],"ndarray":[[0.43782349911420193]]},"meta":{"requestPath":{"classifier":"seldonio/mock_classifier:1.6.0-dev"}}}
{"data":{"names":["proba"],"ndarra

![seldon-rest-dashboard](seldon-rest-dashboard.png)

In [13]:
import time

for i in range(3):
    metric = !curl -s http://localhost:8003/seldon/seldon/rest-seldon/prometheus | grep seldon_api_executor_server_requests_seconds_count
    if metric and len(metric) > 0:
        print(metric[0])
        assert not metric[0] == ""
        break
    else:
        print("Failed to get metrics for rest-seldon")
        time.sleep(2)

for i in range(3):
    metric = !curl -s http://localhost:8003/seldon/seldon/rest-seldon/prometheus | grep seldon_api_executor_server_requests_seconds_summary_count
    if metric and len(metric) > 0:
        print(metric[0])
        assert not metric[0] == ""
        break
    else:
        print("Failed to get metrics for rest-seldon")
        time.sleep(2)

seldon_api_executor_server_requests_seconds_count{code="200",deployment_name="rest-seldon",method="post",predictor_name="model",predictor_version="",service="predictions"} 60
seldon_api_executor_server_requests_seconds_summary_count{code="200",deployment_name="rest-seldon",method="post",predictor_name="model",predictor_version="",service="predictions"} 60


In [14]:
!kubectl delete -f ${RESOURCES}/model_seldon_rest.yaml -n seldon

seldondeployment.machinelearning.seldon.io "rest-seldon" deleted


## Seldon Protocol GRPC Model

In [15]:
!pygmentize ${RESOURCES}/model_seldon_grpc.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: grpc-seldon
spec:
  name: grpcseldon
  protocol: seldon
  transport: grpc
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:1.6.0-dev
          name: classifier
    graph:
      name: classifier
      type: MODEL
      endpoint:
        type: GRPC
    name: model
    replicas: 1


In [16]:
!kubectl apply -f ${RESOURCES}/model_seldon_grpc.yaml -n seldon

seldondeployment.machinelearning.seldon.io/grpc-seldon created


In [17]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=grpc-seldon \
                                 -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "grpc-seldon-model-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
deployment "grpc-seldon-model-0-classifier" successfully rolled out


In [18]:
!cd ../../../executor/proto && for i in `seq 1 60`; do \
 sleep 1 && grpcurl -d '{"data":{"ndarray":[[1.0,2.0]]}}' \
         -rpc-header seldon:grpc-seldon -rpc-header namespace:seldon \
         -plaintext \
         -proto ./prediction.proto  0.0.0.0:8003 seldon.protos.Seldon/Predict; \
done

{
  "meta": {
    "requestPath": {
      "classifier": "seldonio/mock_classifier:1.6.0-dev"
    }
  },
  "data": {
    "names": [
      "proba"
    ],
    "ndarray": [
        [
              0.1951846770138402
            ]
      ]
  }
}
{
  "meta": {
    "requestPath": {
      "classifier": "seldonio/mock_classifier:1.6.0-dev"
    }
  },
  "data": {
    "names": [
      "proba"
    ],
    "ndarray": [
        [
              0.1951846770138402
            ]
      ]
  }
}
{
  "meta": {
    "requestPath": {
      "classifier": "seldonio/mock_classifier:1.6.0-dev"
    }
  },
  "data": {
    "names": [
      "proba"
    ],
    "ndarray": [
        [
              0.1951846770138402
            ]
      ]
  }
}
{
  "meta": {
    "requestPath": {
      "classifier": "seldonio/mock_classifier:1.6.0-dev"
    }
  },
  "data": {
    "names": [
      "proba"
    ],
    "ndarray": [
        [
              0.1951846770138402
            ]
      ]
  }
}
{
  "meta": {
    "requestPath": {
      "cl

![seldon-grpc-dashboard](seldon-grpc-dashboard.png)

In [19]:
for i in range(3):
    metric = !curl -s http://localhost:8003/seldon/seldon/grpc-seldon/prometheus | grep seldon_api_executor_server_requests_seconds_count
    if metric and len(metric) > 0:
        print(metric[0])
        assert not metric[0] == ""
        break
    else:
        print("Failed to get metrics for grpc-seldon")
        time.sleep(2)

for i in range(3):
    metric = !curl -s http://localhost:8003/seldon/seldon/grpc-seldon/prometheus | grep seldon_api_executor_server_requests_seconds_summary_count
    if metric and len(metric) > 0:
        print(metric[0])
        assert not metric[0] == ""
        break
    else:
        print("Failed to get metrics for grpc-seldon")
        time.sleep(2)

seldon_api_executor_server_requests_seconds_count{code="OK",deployment_name="grpc-seldon",method="unary",predictor_name="model",predictor_version="",service="/seldon.protos.Seldon/Predict"} 60
seldon_api_executor_server_requests_seconds_summary_count{code="OK",deployment_name="grpc-seldon",method="unary",predictor_name="model",predictor_version="",service="/seldon.protos.Seldon/Predict"} 60


In [20]:
!kubectl delete -f ${RESOURCES}/model_seldon_grpc.yaml -n seldon

seldondeployment.machinelearning.seldon.io "grpc-seldon" deleted


## Tensorflow Protocol REST Model

In [28]:
!pygmentize ${RESOURCES}/model_tfserving_rest.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: example-tfserving
spec:
  protocol: tensorflow
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - args:
          - --port=8500
          - --rest_api_port=8501
          - --model_name=halfplustwo
          - --model_base_path=gs://seldon-models/tfserving/half_plus_two
          image: tensorflow/serving
          name: halfplustwo
          ports:
          - containerPort: 8501
            name: http
            protocol: TCP
          - containerPort: 8500
            name: grpc
            protocol: TCP
    graph:
      name: halfplustwo
      type: MODEL
      endpoint:
        httpPort: 8501
        grpcPort: 8500
    name: model
    replicas: 1


In [29]:
!kubectl apply -f ${RESOURCES}/model_tfserving_rest.yaml -n seldon

seldondeployment.machinelearning.seldon.io/example-tfserving created


In [32]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=example-tfserving \
                                 -o jsonpath='{.items[0].metadata.name}')

deployment "example-tfserving-model-0-halfplustwo" successfully rolled out


In [34]:
!for i in `seq 1 60`; do \
   sleep 1 && curl -d '{"instances": [1.0, 2.0, 5.0]}' \
   -X POST http://localhost:8003/seldon/seldon/example-tfserving/v1/models/halfplustwo/:predict \
   -H "Content-Type: application/json"; \
done

{
    "predictions": [2.5, 3.0, 4.5
    ]
}{
    "predictions": [2.5, 3.0, 4.5
    ]
}{
    "predictions": [2.5, 3.0, 4.5
    ]
}{
    "predictions": [2.5, 3.0, 4.5
    ]
}{
    "predictions": [2.5, 3.0, 4.5
    ]
}{
    "predictions": [2.5, 3.0, 4.5
    ]
}{
    "predictions": [2.5, 3.0, 4.5
    ]
}{
    "predictions": [2.5, 3.0, 4.5
    ]
}{
    "predictions": [2.5, 3.0, 4.5
    ]
}{
    "predictions": [2.5, 3.0, 4.5
    ]
}{
    "predictions": [2.5, 3.0, 4.5
    ]
}{
    "predictions": [2.5, 3.0, 4.5
    ]
}{
    "predictions": [2.5, 3.0, 4.5
    ]
}{
    "predictions": [2.5, 3.0, 4.5
    ]
}{
    "predictions": [2.5, 3.0, 4.5
    ]
}{
    "predictions": [2.5, 3.0, 4.5
    ]
}{
    "predictions": [2.5, 3.0, 4.5
    ]
}{
    "predictions": [2.5, 3.0, 4.5
    ]
}{
    "predictions": [2.5, 3.0, 4.5
    ]
}{
    "predictions": [2.5, 3.0, 4.5
    ]
}{
    "predictions": [2.5, 3.0, 4.5
    ]
}{
    "predictions": [2.5, 3.0, 4.5
    ]
}{
    "predictions": [2.5, 3.0, 4.5
    ]
}{
    "pred

In [35]:
for i in range(3):
    metric = !curl -s http://localhost:8003/seldon/seldon/rest-tfserving/prometheus | grep seldon_api_executor_server_requests_seconds_count
    if metric and len(metric) > 0:
        print(metric[0])
        assert not metric[0] == ""
        break
    else:
        print("Failed to get metrics for rest-tfserving")
        time.sleep(2)

for i in range(3):
    metric = !curl -s http://localhost:8003/seldon/seldon/rest-tfserving/prometheus | grep seldon_api_executor_server_requests_seconds_summary_count
    if metric and len(metric) > 0:
        print(metric[0])
        assert not metric[0] == ""
        break
    else:
        print("Failed to get metrics for rest-tfserving")
        time.sleep(2)

Failed to get metrics for rest-tfserving
Failed to get metrics for rest-tfserving
Failed to get metrics for rest-tfserving
Failed to get metrics for rest-tfserving
Failed to get metrics for rest-tfserving
Failed to get metrics for rest-tfserving


![tfserving-rest-dashboard](tfserving-rest-dashboard.png)

In [36]:
!kubectl delete -f ${RESOURCES}/model_tfserving_rest.yaml -n seldon

seldondeployment.machinelearning.seldon.io "example-tfserving" deleted


## Tensorflow Protocol GRPC Model

In [37]:
!pygmentize ${RESOURCES}/model_tfserving_grpc.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: grpc-tfserving
spec:
  name: grpctfserving
  protocol: tensorflow
  transport: grpc
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - args: 
          - --port=8500
          - --rest_api_port=8501
          - --model_name=halfplustwo
          - --model_base_path=gs://seldon-models/tfserving/half_plus_two
          image: tensorflow/serving
          name: halfplustwo
          ports:
          - containerPort: 8500
            name: grpc
    graph:
      name: halfplustwo
      type: MODEL
      endpoint:
        service_port: 8500
        type: GRPC
    name: model
    replicas: 1


In [38]:
!kubectl apply -f ${RESOURCES}/model_tfserving_grpc.yaml -n seldon

seldondeployment.machinelearning.seldon.io/grpc-tfserving created


In [39]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=grpc-tfserving \
                                 -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "grpc-tfserving-model-0-halfplustwo" rollout to finish: 0 of 1 updated replicas are available...
deployment "grpc-tfserving-model-0-halfplustwo" successfully rolled out


In [40]:
!cd ../../../executor/proto && for i in `seq 1 60`; do \
 sleep 1 && grpcurl \
   -d '{"model_spec":{"name":"halfplustwo"},"inputs":{"x":{"dtype": 1, "tensor_shape": {"dim":[{"size": 3}]}, "floatVal" : [1.0, 2.0, 3.0]}}}' \
   -rpc-header seldon:grpc-tfserving -rpc-header namespace:seldon \
   -plaintext -proto ./prediction_service.proto \
   0.0.0.0:8003 tensorflow.serving.PredictionService/Predict; \
done

{
  "outputs": {
    "x": {
      "dtype": "DT_FLOAT",
      "tensorShape": {
        "dim": [
          {
            "size": "3"
          }
        ]
      },
      "floatVal": [
        2.5,
        3,
        3.5
      ]
    }
  },
  "modelSpec": {
    "name": "halfplustwo",
    "version": "123",
    "signatureName": "serving_default"
  }
}
{
  "outputs": {
    "x": {
      "dtype": "DT_FLOAT",
      "tensorShape": {
        "dim": [
          {
            "size": "3"
          }
        ]
      },
      "floatVal": [
        2.5,
        3,
        3.5
      ]
    }
  },
  "modelSpec": {
    "name": "halfplustwo",
    "version": "123",
    "signatureName": "serving_default"
  }
}
{
  "outputs": {
    "x": {
      "dtype": "DT_FLOAT",
      "tensorShape": {
        "dim": [
          {
            "size": "3"
          }
        ]
      },
      "floatVal": [
        2.5,
        3,
        3.5
      ]
    }
  },
  "modelSpec": {
    "name": "halfplustwo",
    "version": "123",
 

![dashboard](tfserving-grpc-dashboard.png)

In [41]:
for i in range(3):
    metric = !curl -s http://localhost:8003/seldon/seldon/grpc-tfserving/prometheus | grep seldon_api_executor_server_requests_seconds_count
    if metric and len(metric) > 0:
        print(metric[0])
        assert not metric[0] == ""
        break
    else:
        print("Failed to get metrics for grpc-tfserving")
        time.sleep(2)

for i in range(3):
    metric = !curl -s http://localhost:8003/seldon/seldon/grpc-tfserving/prometheus | grep seldon_api_executor_server_requests_seconds_summary_count
    if metric and len(metric) > 0:
        print(metric[0])
        assert not metric[0] == ""
        break
    else:
        print("Failed to get metrics for grpc-tfserving")
        time.sleep(2)

seldon_api_executor_server_requests_seconds_count{code="OK",deployment_name="grpc-tfserving",method="unary",predictor_name="model",predictor_version="",service="/tensorflow.serving.PredictionService/Predict"} 60
seldon_api_executor_server_requests_seconds_summary_count{code="OK",deployment_name="grpc-tfserving",method="unary",predictor_name="model",predictor_version="",service="/tensorflow.serving.PredictionService/Predict"} 60


In [42]:
!kubectl delete -f ${RESOURCES}/model_tfserving_grpc.yaml -n seldon

seldondeployment.machinelearning.seldon.io "grpc-tfserving" deleted


In [43]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon-system

Context "kind-ansible" modified.


In [44]:
!helm delete seldon-core-analytics -n seldon-system

W0401 17:13:44.115573  612502 warnings.go:70] rbac.authorization.k8s.io/v1beta1 RoleBinding is deprecated in v1.17+, unavailable in v1.22+; use rbac.authorization.k8s.io/v1 RoleBinding
W0401 17:13:44.124540  612502 warnings.go:70] rbac.authorization.k8s.io/v1beta1 Role is deprecated in v1.17+, unavailable in v1.22+; use rbac.authorization.k8s.io/v1 Role
W0401 17:13:44.128226  612502 warnings.go:70] rbac.authorization.k8s.io/v1beta1 ClusterRoleBinding is deprecated in v1.17+, unavailable in v1.22+; use rbac.authorization.k8s.io/v1 ClusterRoleBinding
W0401 17:13:44.128226  612502 warnings.go:70] rbac.authorization.k8s.io/v1beta1 ClusterRoleBinding is deprecated in v1.17+, unavailable in v1.22+; use rbac.authorization.k8s.io/v1 ClusterRoleBinding
W0401 17:13:44.128226  612502 warnings.go:70] rbac.authorization.k8s.io/v1beta1 ClusterRoleBinding is deprecated in v1.17+, unavailable in v1.22+; use rbac.authorization.k8s.io/v1 ClusterRoleBinding
W0401 17:13:44.128226  612502 warnings.go:70] r

In [45]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "kind-ansible" modified.
